# Relation Classifier RNN Model

Our goal here is to implement a relation model. Our goal is to do basic relation classification.

First things first, set up the initial configuration.

In [1]:
import sys
import pandas as pd
import logging
sys.path.insert(0, '../../')

from lib.utils import setup_training

# Create root logger
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

random_seed = 123
device = 0
is_cuda, _ = setup_training(device, random_seed) 
# Async minibatch allocation for speed
# Reference: http://timdettmers.com/2015/03/09/deep-learning-hardware-guide/
# TODO: look into cuda_async device=device
cuda_async = lambda t: t.cuda(async=True) if is_cuda else t  # Use with tensors
cuda = lambda t: t.cuda(device=device) if is_cuda else t  # Use with nn.modules

2018-01-15 10:53:11,397 | INFO : Device: 0
2018-01-15 10:53:11,398 | INFO : Seed: 123


## Dataset

Load our dataset. Log a couple rows.

In [2]:
import os 
from lib.datasets.dataset import Dataset

def simple_qa_predicate(directory='../../data/relation_classifier',
                        train=False,
                        dev=False,
                        test=False,
                        train_filename='train.txt',
                        dev_filename='dev.txt',
                        test_filename='test.txt'):
    """
    Sample Data:
        Input: what is the book e about?
        Output: www.freebase.com/book/written_work/subjects
    """
    ret = []
    datasets = [(train, train_filename), (dev, dev_filename), (test, test_filename)]
    for is_requested, filename in datasets:
        if not is_requested:
            continue
        full_path = os.path.join(directory, filename)
        data = pd.read_table(
            full_path, header=None, names=['question', 'relation'])
        rows = []
        for _, row in data.iterrows():
            rows.append({'text': row['question'].strip(), 'relation': row['relation']})
        ret.append(Dataset(rows))

    if len(ret) == 1:
        return ret[0]
    else:
        return tuple(ret)

In [3]:
train_dataset, dev_dataset = simple_qa_predicate(train=True, dev=True, test=False)

print('Num Training Data: %d' % len(train_dataset))
print('Train Sample:')
display(pd.DataFrame(train_dataset[:5]))
print('Num Development Data: %d' % len(dev_dataset))
print('Development Sample:')
display(pd.DataFrame(dev_dataset[:5]))

Num Training Data: 74520
Train Sample:


,relation,text
0,book/written_work/subjects,what is the book <e> about
1,music/release_track/release,to what release does the release track <e> com...
2,film/film/country,what country was the film <e> from
3,music/producer/tracks_produced,what songs have <e> produced ?
4,music/release/producers,who produced <e> ?


Num Development Data: 10648
Development Sample:


,relation,text
0,symbols/namesake/named_after,who was the <e> named after
1,people/person/place_of_birth,where was <e> born
2,music/release/region,what is a region that <e> was released in
3,film/director/film,what is a film directed by <e> ?
4,music/release/region,what country was <e> released in


## Encode Text

Here we encode our data into a numerical format. 

In [4]:
from lib.text_encoders import IdentityEncoder
from lib.text_encoders import WordEncoder

# NOTE: Given the development vocab might be cheating if we use any statistics from that...
text_encoder = WordEncoder(train_dataset['text'] + dev_dataset['text'], lower=True, append_eos=False)
print('Text encoder vocab size: %d' % text_encoder.vocab_size)
relation_encoder = IdentityEncoder(train_dataset['relation'] + dev_dataset['relation'])
print('Relation encoder vocab size: %d' % relation_encoder.vocab_size)

for dataset in [train_dataset, dev_dataset]:
    for row in dataset:
        row['text'] = text_encoder.encode(row['text'])
        row['relation'] = relation_encoder.encode(row['relation'])

Text encoder vocab size: 5877
Relation encoder vocab size: 1691


# Loss

Instantiate the loss.

In [5]:
from torch.nn.modules.loss import NLLLoss

criterion = cuda(NLLLoss())

## Hyperparameters

Set up the hyperparameters for our model. We use `lib.configurable` to setup arguments.

In [6]:
from torch.optim import Adam
from lib.configurable import configurable
from lib.configurable import add_config
from lib.configurable import log_config


Adam.__init__ = configurable(Adam.__init__)

embedding_size = 300

SIMPLE_PREDICATE_HYPERPARAMETERS = {
    'lib': {
        'nn.seq_to_label.SeqToLabel.__init__': {
            'bidirectional': True,
            'embedding_size': embedding_size,
            'rnn_size': 300,
            'freeze_embeddings': True,
            'rnn_cell': 'lstm',
            'decode_dropout': 0.3,  # dropout before fully connected layer in RNN
            'rnn_layers': 2,
            'rnn_variational_dropout': 0.3,
            'embedding_dropout': 0.3
        },
        'optim.Optimizer.__init__.max_grad_norm': None,
    },
    'torch.optim.adam.Adam.__init__': {
        'lr': 1e-4,
        'weight_decay': 0,
    }
}

add_config(SIMPLE_PREDICATE_HYPERPARAMETERS)
log_config()

2018-01-15 10:53:17,350 | INFO : Checking configuration...
2018-01-15 10:53:17,355 | INFO : Configuration checked.
2018-01-15 10:53:17,355 | INFO : Global configuration:
2018-01-15 10:53:17,357 | INFO : {   'lib': {   'nn': {   'seq_to_label': {   'SeqToLabel': {   '__init__': {   'bidirectional': True,
                                                                               'decode_dropout': 0.3,
                                                                               'embedding_dropout': 0.3,
                                                                               'embedding_size': 300,
                                                                               'freeze_embeddings': True,
                                                                               'rnn_cell': 'lstm',
                                                                               'rnn_layers': 2,
                                                                               'rnn_s

# Model

Instantiate the model.

In [7]:
import torch
from lib.pretrained_embeddings import FastText

# Load embeddings
unk_init = lambda t: torch.FloatTensor(t).uniform_(-0.1, 0.1)
pretrained_embedding = FastText(language='en', cache='./../../.pretrained_embeddings_cache')
embedding_weights = torch.Tensor(text_encoder.vocab_size, pretrained_embedding.dim)
for i, token in enumerate(text_encoder.vocab):
    embedding_weights[i] = pretrained_embedding[token]

2018-01-15 10:53:17,364 | INFO : Loading vectors from ./../../.pretrained_embeddings_cache/wiki.en.vec.pt


In [8]:
from lib.nn import SeqToLabel

def make_model():
    model = SeqToLabel(text_encoder.vocab_size, relation_encoder.vocab_size)
    for param in model.parameters():
        param.data.uniform_(-0.1, 0.1)

    model.encoder.embedding.weight.data.copy_(embedding_weights)

    cuda(model)
    return model

## Gradient Descent Optimizer 

Instantiate the gradient descent optimizer.

In [9]:
from lib.optim import Optimizer

# https://github.com/pytorch/pytorch/issues/679
def make_optimizer():
    params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = Optimizer(Adam(params=params))
    return optimizer

## Training Loop

Below here, we do a training loop over a number of epochs.

In [10]:
import torch
from torch.autograd import Variable
from lib.utils import pad_batch

# Defines how to combine a batch of rows into a tensor
def collate_fn(batch, train=True):
    """ list of tensors to a batch variable """
    # PyTorch RNN requires sorting decreasing size
    batch = sorted(batch, key=lambda row: len(row['text']), reverse=True)
    input_batch, input_lengths = pad_batch([row['text'] for row in batch])
    relations = [row['relation'] for row in batch]

    # PyTorch RNN requires batches to be transposed for speed and integration with CUDA
    to_variable = (
        lambda b: Variable(torch.stack(b).t_().squeeze(0).contiguous(), volatile=not train))

    return (to_variable(input_batch), torch.LongTensor(input_lengths), to_variable(relations))

In [11]:
from lib.samplers import BucketBatchSampler
from lib.samplers import SortedSampler


sort_key = lambda r: r['text'].size()[0]
def get_train_iterator():
    # Use bucket sampling to group similar sized text but with noise + random
    batch_sampler = BucketBatchSampler(train_dataset, sort_key, train_max_batch_size)
    return DataLoader(
        train_dataset,
        batch_sampler=batch_sampler,
        collate_fn=collate_fn,
        pin_memory=is_cuda,
        num_workers=0)


def get_dev_iterator():
    # Use sorted sampler for speed
    return DataLoader(
        dev_dataset,
        batch_size=dev_max_batch_size,
        sampler=SortedSampler(dev_dataset, sort_key, sort_noise=0.0),
        collate_fn=partial(collate_fn, train=False),
        pin_memory=is_cuda,
        num_workers=0)

In [12]:
from lib.utils import get_total_parameters
from lib.utils import get_log_directory_path

epochs = 50
train_max_batch_size = 16
dev_max_batch_size = 128
log_directory = get_log_directory_path('relation_classifier', './../../logs')
model = make_model()
optimizer = make_optimizer()

print('Log Directory: %s' % log_directory)
print('Devevelopment Max Batch Size: %s' % dev_max_batch_size)
print('Train Max Batch Size: %s' % train_max_batch_size)
print('Epochs: %s' % epochs)
print('Total Parameters: %d' % get_total_parameters(model))
print('Model:\n%s' % model)

2018-01-15 10:53:19,943 | INFO : seq_to_label.SeqToLabel.__init__ was configured with:
{   'bidirectional': True,
    'decode_dropout': 0.3,
    'embedding_dropout': 0.3,
    'embedding_size': 300,
    'freeze_embeddings': True,
    'input_vocab_size': 5877,
    'output_vocab_size': 1691,
    'rnn_cell': 'lstm',
    'rnn_layers': 2,
    'rnn_size': 300,
    'rnn_variational_dropout': 0.3}
2018-01-15 10:53:19,944 | INFO : seq_encoder.SeqEncoder.__init__ no config for: lib.nn.seq_encoder.SeqEncoder.__init__
2018-01-15 10:53:19,945 | INFO : seq_encoder.SeqEncoder.__init__ was configured with:
{   'bidirectional': True,
    'embedding_dropout': 0.3,
    'embedding_size': 300,
    'freeze_embeddings': True,
    'n_layers': 2,
    'rnn_cell': 'lstm',
    'rnn_size': 300,
    'rnn_variational_dropout': 0.3,
    'vocab_size': 5877}
2018-01-15 10:53:22,091 | INFO : adam.Adam.__init__ was configured with:
{'lr': 0.0001, 'params': <filter object at 0x7fc1c459c898>, 'weight_decay': 0}
2018-01-15 1

In [13]:
from tqdm import tqdm_notebook

from torch.utils.data import DataLoader
from functools import partial

from lib.metrics import get_accuracy
from lib.metrics import get_accuracy_top_k
from lib.metrics import print_random_sample
from lib.checkpoint import Checkpoint

# TODO: Try to concat multiple templated questions together and do a multi label classifier.


# Train!
for epoch in range(epochs):
    print('Epoch %d' % epoch)
    
    # Iterate over the training data
    model.train(mode=True)
    train_iterator = get_train_iterator()
    for text, text_lengths, relation in tqdm_notebook(train_iterator):
        optimizer.zero_grad()
        relation = cuda_async(relation)
        output = model(cuda_async(text), cuda_async(text_lengths))
        loss = criterion(output, relation)

        # Backward propagation
        loss.backward()
        optimizer.step()

    # Save checkpoint
    Checkpoint.save(
        log_directory,
        {
            'model': model,
            'optimizer': optimizer,
            'relation_encoder': relation_encoder,
            'text_encoder': text_encoder
        },
        device=device)

    # Evaluate
    model.train(mode=False)
    texts, relations, outputs = [], [], []
    total_loss = 0
    dev_iterator = get_dev_iterator()
    for text, text_lengths, relation in tqdm_notebook(dev_iterator):
        relation = cuda_async(relation)
        output = model(cuda_async(text), cuda_async(text_lengths))
        
        # Compute metrics
        total_loss += criterion(output, relation).data[0] * relation.size()[0]
        # Prevent memory leak by moving output from variable to tensor
        texts.extend(text.data.cpu().transpose(0, 1).split(split_size=1, dim=0))
        relations.extend(relation.data.cpu().split(split_size=1, dim=0))
        outputs.extend(output.data.cpu().split(split_size=1, dim=0))

    # Print metrics
    optimizer.update(total_loss / len(dev_dataset), epoch)
    print_random_sample(texts, relations, outputs, text_encoder, relation_encoder, n_samples=5)
    print('Loss: %.03f' % (total_loss / len(dev_dataset)))
    get_accuracy(relations, outputs, print_=True) # TODO: These should be the same function
    get_accuracy_top_k(relations, outputs, k=3, print_=True)
    get_accuracy_top_k(relations, outputs, k=5, print_=True)
    print()

Epoch 0


HBox(children=(IntProgress(value=0, max=4658), HTML(value='')))

../../lib/nn/seq_to_label.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  scores = F.log_softmax(output)



2018-01-15 10:53:49,306 | INFO : Saving checkpoint 01m_15d_10h_53m_49s.pt


HBox(children=(IntProgress(value=0, max=84), HTML(value='')))

...


2018-01-15 11:14:30,862 | INFO : Random Sample:

Positive Samples:
                                 Source                      Target  \
0             what film did <e> produce          film/producer/film   
1    what artist produced the album <e>          music/album/artist   
2           what is the gender of <e> ?        people/person/gender   
3                           what is <e>  common/topic/notable_types   
4  which country is <e> originally from   people/person/nationality   

                   Prediction  
0          film/producer/film  
1          music/album/artist  
2        people/person/gender  
3  common/topic/notable_types  
4   people/person/nationality  

Negative Samples:
                                           Source  \
0                         what language is <e> in   
1  <e> is considered a part of what genre ? <pad>   
2                         where play <e> 's music   
3                        which group released <e>   
4                 what is a s

HBox(children=(IntProgress(value=0, max=4658), HTML(value='')))


2018-01-15 11:14:55,104 | INFO : Saving checkpoint 01m_15d_11h_14m_55s.pt


HBox(children=(IntProgress(value=0, max=84), HTML(value='')))


2018-01-15 11:14:55,634 | INFO : Random Sample:

Positive Samples:
                                           Source  \
0  who uploaded the content about <e> to freebase   
1                  what was involved in the <e> ?   
2               which country was <e> released in   
3         what is a musician that plays <e> music   
4               what language was in the film <e>   

                                     Target  \
0                  type/content/uploaded_by   
1  base/culturalevent/event/entity_involved   
2                      music/release/region   
3                       music/genre/artists   
4                        film/film/language   

                                 Prediction  
0                  type/content/uploaded_by  
1  base/culturalevent/event/entity_involved  
2                      music/release/region  
3                       music/genre/artists  
4                        film/film/language  

Negative Samples:
                                   